<a href="https://colab.research.google.com/github/lucasabbruzzini/Portfolio/blob/main/Mortgage.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Implementation of Mortgage Loan process from the lending perspective

# Setup

In [ ]:
%%capture
import locale
locale.setlocale(locale.LC_ALL, 'en_US.UTF-8')
import numpy as np
import pandas as pd

from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
from sklearn.metrics import accuracy_score

In [ ]:
%%capture

# Defining Functions

def get_index(llpa_table, fico_score, l2v):
#This funtions works as a lookup function to get the row and column indexes for a given LTV and Fico Scores

  for i in range(1, len(llpa_table.index)):
    if fico_score > llpa_table.index[i] and fico_score <= llpa_table.index[i - 1]:
      fico_index = i - 1
      break
    else:
      fico_index = i

  #Finding the range for the clients LTV

  for i in range(1, len(llpa_table.columns)):
    if l2v < llpa_table.columns[i] and l2v >= llpa_table.columns[i - 1]:
      l2v_index = i - 1
      break
    else:
      l2v_index = i
  return (fico_index, l2v_index)


def calc(application, mortgage_interest, llpa_table, parameters_l2v, parameters_d2i):
  #This function gets the applicants information, llpa matrix and firms parameters to calculate the maximun loan approved
  #interest rates, L2V, DTI and monthly payments

  interest = {}
  loan = {}
  d2i = {}

  to_borrow = (application['ask_price'] - application['down'])
  l2v = round(to_borrow/application['appraised'],2)
  parameters_l2v.update({'proposed': l2v})

  years = ['15', '30']
  terms = ['proposed', 'max', 'desired']
  monthly = {}

  for year in years:
    for term in terms:
      aux = term + '-' + str(year)
      y = int(year)

      #Here it calculates the interest for a given scenario based on average interest and llpa
      interest[aux] = mortgage_interest[year] + llpa_table.iloc[get_index(llpa_table, application['fico_score'], parameters_l2v[term])]

      print('\n')
      if term == 'proposed':
        #Here it callculates the monthly payments and the DTI for the proposed mortgage by the client
        monthly[aux] = (to_borrow)*interest[aux]*(1 + interest[aux])**(y)/((1+  interest[aux])**(y) - 1)/12
        d2i[aux] = (application['expenses'] + monthly[aux])/application['income']

      else:
        #Here it calculate 2 scenarios for mortgage, desired and maximun, the desired means the maximum amount the applicant can to_borrow
        #without having to pay mortgage insurance, and the maximun is the maximun amount the applicant can to_borrow with insurance
        #they are based on the banks parametrs
        monthly[aux] = application['income']*parameters_d2i[term] - application['expenses']
        loan[aux] = 12*monthly[aux]/(interest[aux]*(1 + interest[aux])**(y)/((1+  interest[aux])**(y) - 1))

  parameters_d2i.update(d2i)

  loan.update({'proposed': to_borrow})

  for key, value in interest.items():
      interest[key] = value * 100

  dicts = [parameters_l2v, parameters_d2i, loan, monthly, interest]

  for my_dict in dicts:
    for key, value in my_dict.items():
      my_dict[key] = round(value, 2)

  return parameters_l2v, parameters_d2i, loan, interest, monthly


def eval(loan, l2v, d2i):
  #This funtion compare the values from LTV, DTI and loan approved

    # Check
  # 0 - Below bank's desired amount
  # 1 - Above bank's desired amount but below maximum allowed - required insurance
  # 2 - Above bank's maximum allowed - mortgage denied

  l2v_check = 0

  d2i_check ={}
  d2i_check['30'] = 0
  d2i_check['15'] = 0

  loan_check = {}
  loan_check['30'] = 0
  loan_check['15'] = 0

  years = ['15', '30']
  terms = ['max', 'desired']

  # Checking L2V

  for term in terms:
      if l2v['proposed'] > l2v[term]:
        l2v_check = l2v_check+ 1

  #Checking d2i
  for year in years:
    for term in terms:
      aux = 'proposed' + '-' + str(year)
      if d2i[aux] > d2i[term]:
        d2i_check[year] = d2i_check[year] +1

  #Checking Loan
  for year in years:
    for term in terms:
      if loan['proposed'] > loan[term+'-'+year]:
        loan_check[year] = loan_check[year] +1

  check = {}
  check = {'l2v': l2v_check,
           'd2i-30': d2i_check['30'],
           'd2i-15': d2i_check['15'],
           'loan-30': loan_check['30'],
           'loan-15': loan_check['15']
           }

  return check




# Mortgage

## Firms Parameters

In [ ]:
### Parameters Setup

# Mortgage rates from https://www.freddiemac.com/pmms in Febrary 1st
mortgage_interest = {}
mortgage_interest['15'] = 0.0596 + 0.01 # Current Yearly interest rate for 15 year mortage + 1% for operation and margin
mortgage_interest['30'] = 0.0669 + 0.01 # Current Yearly interest rate for 30 year mortage + 1% for operation and margin

#Weights set by the frim for risk management
weights = {
            'appraisal_val': 0.2,
            'credit': 0.2,
            'l2v': 0.2,
            'd2i': 0.2,
            'stability': 0.2
}

#Bank's paramters of LTV and DTI, collected as market standard
parameters_l2v = {'max': 0.90,
                  'desired': 0.75}
parameters_d2i = {'max': 0.45,
                  'desired': 0.36}

In [ ]:
#LLPA data extract manually from https://singlefamily.fanniemae.com/media/9391/display

#The following will construct the LLPA Matrix, this table uses LTV (columns) and FICO scores (rows) to calculate the LLPA to be added to the interest rates
llpa_data =[
            [0,0,0,0,0.00375,0.00375,0.0025,0.0025,0.00125],
            [0,0,0,0.0025,0.00625,0.00625,0.005,0.005,0.0025],
            [0,0,0.00125,0.00375,0.00875,0.01,0.0075,0.00625,0.005],
            [0,0,0.0025,0.0075,0.0125,0.0125,0.01,0.00875,0.0075],
            [0,0,0.00375,0.00875,0.01375,0.015,0.0125,0.01125,0.00875],
            [0,0,0.00625,0.01125,0.0175,0.01875,0.015,0.01375,0.01125],
            [0,0,0.0075,0.01375,0.01875,0.02125,0.0175,0.01625,0.0125],
            [0,0,0.01125,0.015,0.0225,0.025,0.02,0.01875,0.015],
            [0,0.00125,0.015,0.02125,0.0275,0.02875,0.02625,0.0225,0.0175]
            ]

l2v_ratios = [0.3,0.6,0.7,0.75,0.8,0.85,0.9,0.95,1.01]
credit_ranges = [851,779,759,739,719,699,679,659,639]
llpa_table = pd.DataFrame(llpa_data, columns = l2v_ratios, index = credit_ranges)

llpa_table

,0.30,0.60,0.70,0.75,0.80,0.85,0.90,0.95,1.01
851,0,0.00000,0.00000,0.00000,0.00375,0.00375,0.00250,0.00250,0.00125
779,0,0.00000,0.00000,0.00250,0.00625,0.00625,0.00500,0.00500,0.00250
759,0,0.00000,0.00125,0.00375,0.00875,0.01000,0.00750,0.00625,0.00500
739,0,0.00000,0.00250,0.00750,0.01250,0.01250,0.01000,0.00875,0.00750
719,0,0.00000,0.00375,0.00875,0.01375,0.01500,0.01250,0.01125,0.00875
699,0,0.00000,0.00625,0.01125,0.01750,0.01875,0.01500,0.01375,0.01125
679,0,0.00000,0.00750,0.01375,0.01875,0.02125,0.01750,0.01625,0.01250
659,0,0.00000,0.01125,0.01500,0.02250,0.02500,0.02000,0.01875,0.01500
639,0,0.00125,0.01500,0.02125,0.02750,0.02875,0.02625,0.02250,0.01750


## Initial Application
The first step is for the prospective borrower to submit a mortgage loan application. This application includes personal and financial information such as:
- Income
- Employment history
- Credit history/Credit Score
- Amount they wish to borrow (asking price)


For the purpose of this assignment we will consider applications for mortgages made by a household with only one income
In this assignment we will not be evaluating: Age, Gender, Marital Status, Educational Level.
For simplicity taxes and fees will not be added to monthly payments but they are taken into consideration for example when assessing Debit to Income (DTI)


In [ ]:
# Here are the inputs on the clients application

application = {}
application['income'] = 10000 # in USD/month
application['fico_score'] = 750 #  fico scores
application['ask_price'] =  400000 #  amount they wish to borrow in USD
application['expenses'] = 1000 #  monthly expenses in USD
application['down'] =application['ask_price']*0.2  #  expected down payment
application['appraised'] = 450000 #  appraisal value of the property

employment = {}

employment['last'] = 0 #  years employed in the current employer
employment['qtd'] = 0 #  number of employers in the past 10 years
employment['total'] = 0 #  number of years employed in the past 10 years


## Documentation Review
Our underwriters will review the borrower's documentation, which may include:
- Pay stubs
- Tax returns
- Bank statements
- Credit reports
This is done to assess the borrower's creditworthiness and ability to repay the loan.

For the purpose of this activity this part will assume all documents are valid

## Mortgage Analysis

Credit Score
- Credit scores play a significant role in the mortgage lending process. We use credit scores to gauge the borrower's credit risk. A higher credit score indicates a lower credit risk, while a lower score may indicate a higher risk.
- For the purpose of this assignment the fico score will be evaluated as:
 - 800 to 850: Excellent. Individuals in this range are considered to be - low-risk borrowers:
 - 740 to 799: Very good
 - 670 to 739: Good
 - 580 to 669: Fair
 - 300 to 579: Poor
- FICO scores also take into consideration Payment History, Credit Utilization, Len ght of Credit History, Types of Credit Accouts and Recent Credit Inquiries


Appraisal of the Property
- An independent appraiser evaluates the property's value to ensure that it's worth at least the loan amount requested. The property serves as collateral for the mortgage.


Debt-to-Income Ratio (DTI)
- We calculate the borrower's DTI by comparing their total monthly debt obligations (including the mortgage payment) to their gross monthly income. A lower DTI is preferable, as it signifies the borrower has more disposable income to cover their mortgage payments.


$DTI = \frac{Expenses + ExpectedMortgage}{Income}$


Loan-to-Value Ratio (LTV)
- LTV is the ratio of the loan amount to the appraised value of the property. We typically have maximum LTV limits, and borrowers may need to make a larger down payment if their LTV exceeds these limits. Lower LTV ratios are generally less risky.



$LTV = \frac{AskingPrice - DownPayment}{AppraisedValue}$


Employment and Income Stability
- We assess the stability of the borrower's employment and income. Consistent employment and a reliable source of income are positive indicators.


Loan Terms
- It will be considered Fixed Rate for both 15 Years and 30 Years Loan

Interest Rates
- Interest Rates will be calculated as follow:

  $interest = Average Mortgage Rate + LLPA + Operating Costs +  Profit Margin$

  where:
  - Average Mortgage Rate is the 30-Year (15 Year) Fixed Rate Mortgage Average in the United States
  - LLPA is the Loan-Level Price Adjustment based on L2V and Fico Scores and will be used as the Risk Premium
  - Operating Costs and Profit Margin are arbitrary for the lender's loan operating costs and target profit margin

Loan Amount
- To calculate the maximum loan amount aproved we use the following formula

$ApprovedAmount = \frac{Payments}{\frac{r(1+r)^t}{(1+r)^t - 1}}$

  - where:
    - r is the interest rate for period
    - Payments are the payments for the period (Monthly, Yearly)
    - t is the time (years, months)

- We calculate the above for the 30 year and 15 periods and also for the maximun conditions set by the firm for LTV and DTI


In [ ]:
#Here we calculate the LTV, DTI, LOAN, INTEREST and MONTHLY payments based on the funtions above

l2v, d2i, loan, interest, monthly = calc(application, mortgage_interest, llpa_table, parameters_l2v, parameters_d2i)
check = eval(loan, l2v, d2i)

In [ ]:
# aux for printing decision
verification = {}
aproval = {}
verification['30'] = [value for key, value in check.items() if key.endswith('30')]
verification['15'] = [value for key, value in check.items() if key.endswith('15')]
years = ['15','30']
for year in years:
  if max(verification[year]) == 0:
    aproval[year] = "APPROVED"
  if max(verification[year]) == 1:
    aproval[year] = "APPROVED but required Mortgage Insurance"
  if max(verification[year]) == 2:
    aproval[year] = "DENIED"


In [ ]:
print('For the applications conditions as follow:')
print('Applicant FICO Score:', application['fico_score'])
print('Applicant monthly Income:', locale.currency(application['income'], grouping=True))
print('Applicant monthly Expenses:', locale.currency(application['expenses'], grouping=True))
print('Home asking price:', locale.currency(application['ask_price'], grouping=True))
print('Proposed Down Payment:', locale.currency(application['down'], grouping=True))
print('Home appraised value:', locale.currency(application['appraised'], grouping=True))
print('Required Loan Amount:', locale.currency(loan['proposed'], grouping=True))
print('\n')


print('Our analysis resulted in the following:')
print('Loan to Value (LTV):', l2v['proposed'], '(bank\'s policy insurance free below', parameters_l2v['desired'], 'and', parameters_l2v['max'], 'maximun with insurance)')

years = ['30', '15']

for year in years:
  aux = 'proposed-'+year
  print('\n')
  print('For a', year, 'Year Fixed Rate Mortgage the Loan was', aproval[year], 'considering the following conditions:')
  print('Debt to Income (DTI):', d2i[aux], '(bank\'s policy insurance free below', parameters_d2i['desired'], 'and', parameters_d2i['max'], 'maximum with insurance)')
  print('With a monthly payment of:', locale.currency(monthly[aux], grouping=True))
  print('Interest:', interest[aux],'% APR')
  print('The maximum amount approved for a', year, 'Year Fixed Mortgage under the given conditions are',  locale.currency(loan['desired-'+str(year)], grouping=True), '(', interest['desired-'+str(year)],'% interest rate) without insurance and', locale.currency(loan['max-'+str(year)], grouping=True), '(', interest['max-'+str(year)],'% interest rate) with insurance')



For the applications conditions as follow:
Applicant FICO Score: 750
Applicant monthly Income: $10,000.00
Applicant monthly Expenses: $1,000.00
Home asking price: $400,000.00
Proposed Down Payment: $80,000.00
Home appraised value: $450,000.00
Required Loan Amount: $320,000.00


Our analysis resulted in the following:
Loan to Value (LTV): 0.71 (bank's policy insurance free below 0.75 and 0.9 maximun with insurance)


For a 30 Year Fixed Rate Mortgage the Loan was APPROVED considering the following conditions:
Debt to Income (DTI): 0.33 (bank's policy insurance free below 0.36 and 0.45 maximum with insurance)
With a monthly payment of: $2,327.51
Interest: 7.82 % APR
The maximum amount approved for a 30 Year Fixed Mortgage under the given conditions are $349,099.70 ( 8.06 % interest rate) without insurance and $453,855.79 ( 8.44 % interest rate) with insurance


For a 15 Year Fixed Rate Mortgage the Loan was APPROVED but required Mortgage Insurance considering the following conditions:
De

# Risk Management

Assuming the mortgage was ACCEPTED with/without insurance we proceed to the Risk Management part which will consist on the follow:

- Analyse the risk for this mortgage in our portfolio
- Analyse alternatives to minimize the risk such as MBS

### Risk Assessment

In order to evaluate the probability of default, the following data set was downloaded:

http://www.creditriskanalytics.net/datasets-private2.html

The data set mortgage is in panel form and reports origination and performance observations for 50,000 residential U.S. mortgage borrowers over 60 periods. The periods have been deidentified. As in the real world, loans may originate before the start of the observation period (this is an issue where loans are transferred between banks and investors as in securitization). The loan observations may thus be censored as the loans mature or borrowers refinance. The data set is a randomized selection of mortgage-loan-level data collected from the portfolios underlying U.S. residential mortgage-backed securities (RMBS) securitization portfolios and provided by International Financial Research (www.internationalfinancialresearch.org).

 id: Borrower ID
- time: Time stamp of observation
- orig_time: Time stamp for origination
- first_time: Time stamp for first observation
- mat_time: Time stamp for maturity
- balance_time: Outstanding balance at observation time
- LTV_time: Loan-to-value ratio at observation time, in %
- interest_rate_time: Interest rate at observation time, in %
- hpi_time: House price index at observation time, base year = 100
- gdp_time: Gross domestic product (GDP) growth at observation time, in %
- uer_time: Unemployment rate at observation time, in %
- REtype_CO_orig_time: Real estate type condominium = 1, otherwise = 0
- REtype_PU_orig_time: Real estate type planned urban development = 1, otherwise = 0
- REtype_SF_orig_time: Single-family home = 1, otherwise = 0
- investor_orig_time: Investor borrower = 1, otherwise = 0
- balance_orig_time: Outstanding balance at origination time
- FICO_orig_time: FICO score at origination time, in %
- LTV_orig_time: Loan-to-value ratio at origination time, in %
- Interest_Rate_orig_time: Interest rate at origination time, in %
- hpi_orig_time: House price index at origination time, base year = 100
- default_time: Default observation at observation time
- payoff_time: Payoff observation at observation time
- status_time: Default (1), payoff (2), and nondefault/nonpayoff (0) observation at observation time

In [ ]:
url = 'https://drive.google.com/uc?export=download&id=1JqKMrtHByum91grL_huVah6Aeh3o6wTu'
df = pd.read_csv(url)
df.drop(['REtype_CO_orig_time','REtype_PU_orig_time','REtype_SF_orig_time','payoff_time', 'status_time', 'investor_orig_time'], axis=1, inplace=True)
df.set_index('id', inplace = True)
df = df.dropna()
df.describe()

,time,orig_time,first_time,mat_time,balance_time,LTV_time,interest_rate_time,hpi_time,gdp_time,uer_time,balance_orig_time,FICO_orig_time,LTV_orig_time,Interest_Rate_orig_time,hpi_orig_time,default_time
count,622219.000000,622219.000000,622219.000000,622219.000000,6.222190e+05,622219.000000,622219.000000,622219.000000,622219.000000,622219.000000,6.222190e+05,622219.000000,622219.000000,622219.000000,622219.000000,622219.000000
mean,35.799309,20.576450,24.606801,137.213671,2.460523e+05,83.078239,6.700836,184.102317,1.381058,6.517258,2.563651e+05,673.668734,78.976433,5.650538,198.148398,0.024353
std,11.580002,7.610427,6.739667,19.530448,2.141613e+05,24.968839,2.077520,27.650825,1.964707,1.921405,2.197009e+05,71.690396,10.127634,2.976440,33.934437,0.154143
min,1.000000,-40.000000,1.000000,18.000000,0.000000e+00,0.000000,0.000000,107.830000,-4.146711,3.800000,4.791500e+03,400.000000,50.100000,0.000000,75.710000,0.000000
25%,27.000000,18.000000,21.000000,137.000000,1.021175e+05,67.109446,5.650000,158.600000,1.104163,4.700000,1.080000e+05,626.000000,75.000000,5.000000,179.450000,0.000000
50%,34.000000,22.000000,25.000000,142.000000,1.807573e+05,82.253269,6.625000,180.520000,1.850689,5.700000,1.880000e+05,678.000000,80.000000,6.290000,216.770000,0.000000
75%,44.000000,25.000000,28.000000,145.000000,3.375570e+05,100.634381,7.875000,212.730000,2.694111,8.200000,3.520000e+05,729.000000,80.000000,7.451500,222.390000,0.000000
max,60.000000,60.000000,60.000000,229.000000,8.701859e+06,803.513550,37.500000,226.290000,5.132464,10.000000,8.000000e+06,840.000000,218.500000,19.750000,226.290000,1.000000


In [ ]:
df.tail()

,time,orig_time,first_time,mat_time,balance_time,LTV_time,interest_rate_time,hpi_time,gdp_time,uer_time,balance_orig_time,FICO_orig_time,LTV_orig_time,Interest_Rate_orig_time,hpi_orig_time,default_time
id,,,,,,,,,,,,,,,,
50000,56,16,52,177,517107.42,57.659403,2.664,181.43,1.717053,6.6,664000.0,653,80.0,1.75,167.91,0
50000,57,16,52,177,512274.57,55.359916,2.652,187.20,2.556052,6.2,664000.0,653,80.0,1.75,167.91,0
50000,58,16,52,177,507396.72,54.492206,2.644,188.37,2.868594,6.2,664000.0,653,80.0,1.75,167.91,0
50000,59,16,52,177,502478.87,54.171106,2.638,187.65,2.443648,5.7,664000.0,653,80.0,1.75,167.91,0
50000,60,16,52,177,497521.35,53.023479,2.635,189.82,2.836358,5.7,664000.0,653,80.0,1.75,167.91,0


In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 622219 entries, 1 to 50000
Data columns (total 16 columns):
 #   Column                   Non-Null Count   Dtype  
---  ------                   --------------   -----  
 0   time                     622219 non-null  int64  
 1   orig_time                622219 non-null  int64  
 2   first_time               622219 non-null  int64  
 3   mat_time                 622219 non-null  int64  
 4   balance_time             622219 non-null  float64
 5   LTV_time                 622219 non-null  float64
 6   interest_rate_time       622219 non-null  float64
 7   hpi_time                 622219 non-null  float64
 8   gdp_time                 622219 non-null  float64
 9   uer_time                 622219 non-null  float64
 10  balance_orig_time        622219 non-null  float64
 11  FICO_orig_time           622219 non-null  int64  
 12  LTV_orig_time            622219 non-null  float64
 13  Interest_Rate_orig_time  622219 non-null  float64
 14  hpi_o

In [ ]:
df.tail()

,time,orig_time,first_time,mat_time,balance_time,LTV_time,interest_rate_time,hpi_time,gdp_time,uer_time,balance_orig_time,FICO_orig_time,LTV_orig_time,Interest_Rate_orig_time,hpi_orig_time,default_time
id,,,,,,,,,,,,,,,,
50000,56,16,52,177,517107.42,57.659403,2.664,181.43,1.717053,6.6,664000.0,653,80.0,1.75,167.91,0
50000,57,16,52,177,512274.57,55.359916,2.652,187.20,2.556052,6.2,664000.0,653,80.0,1.75,167.91,0
50000,58,16,52,177,507396.72,54.492206,2.644,188.37,2.868594,6.2,664000.0,653,80.0,1.75,167.91,0
50000,59,16,52,177,502478.87,54.171106,2.638,187.65,2.443648,5.7,664000.0,653,80.0,1.75,167.91,0
50000,60,16,52,177,497521.35,53.023479,2.635,189.82,2.836358,5.7,664000.0,653,80.0,1.75,167.91,0


In [ ]:
X = df.drop('default_time', axis=1)
y = df[['default_time']]
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=.3, random_state=0)
mortgage_logsitcregression = LogisticRegression()
mortgage_logsitcregression.fit(X_train, y_train)
y_pred = mortgage_logsitcregression.predict(X_test)
print('Accuracy of logistic regression classifier on test set: {:.2f}'.format(accuracy_score(y_test, y_pred)))

/usr/local/lib/python3.10/dist-packages/sklearn/utils/validation.py:1143: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


Accuracy of logistic regression classifier on test set: 0.98


In [ ]:
#The max time is 60, so we will set current time as time.
# by analyzing it it can be infered that 1 period is equal to 1 quarter
#so a 30 year mortgage will have 120 periods and a 15 year mortgage will have 60 periods

#Macroeconomic metrics we will use the value found in the last row

time = 61
hpi = 189.82
gdp = 2.836358
uer = 5.7

fields_dict = {'time': time,
               'orig_time': time,
               'first_time': time,
               'mat_time': 180,
               'balance_time': loan['proposed'],
               'LTV_time': l2v['proposed'],
               'interest_rate_time': interest['proposed-30'],
               'hpi_time': hpi,
               'gdp_time': gdp,
               'uer_time': uer,
               'balance_orig_time': loan['proposed'],
               'FICO_orig_time': application['fico_score'],
               'LTV_orig_time': l2v['proposed'],
                'Interest_Rate_orig_time': interest['proposed-30'],
                'hpi_orig_time': hpi}

loan2predict30 = pd.DataFrame(fields_dict, index=range(len(fields_dict)))


In [ ]:
prob_default = mortgage_logsitcregression.predict_proba(loan2predict30)[:,1][0]
print('The probability of default of the loan is', round(prob_default*100,2),'%')

The probability of default of the loan is 1.92 %


In [ ]:
# Since this dataset does not include recovery data we will use a set value for recovery rate of 43.9%,
# given that this is a secure loan, having the house as colateral this can be seen as a good estimate
# value comes from the link below
# https://www.eba.europa.eu/sites/default/files/document_library/About%20Us/Missions%20and%20tasks/Call%20for%20Advice/2020/Report%20on%20the%20benchmarking%20of%20national%20loan%20enforcement%20frameworks/962022/Report%20on%20the%20benchmarking%20of%20national%20loan%20enforcement%20frameworks.pdf

recovery_rate = 0.439
lgd = 1 - recovery_rate

# For EAD we will create a new column to calculate the ccf, by dividing the outstanding balance by the orginal loan

default_df = df.drop(df[df["default_time"] == 0].index)
default_df['ccf'] = default_df['balance_time'] /default_df['balance_orig_time']
default_df['ccf'].describe()
ccf = default_df['ccf'].mean() #Since this is a small dataset we will just take the mean of the ccf and not do any regression for predicting value

# we can then use the ccf to calculate the EAD

ead = ccf*loan['proposed']

# We then calculate the Expected Loss

el = ead*ccf*prob_default

for year in years:
  print('For the', year, 'Year Fixed Rate Mortgage Loan of', locale.currency(loan['proposed'], grouping=True), 'we were able to assess the folowing risk:')
  print('Probability of Default(PD):', round(prob_default*100,2),'%')
  print('Loss Given Default (LGD):', round(lgd*100,2),'%')
  print('Exposure at Default (EAD):', locale.currency(ead, grouping=True))
  print('Expected Loss (LD):', locale.currency(el, grouping=True))
  print('\n')

For the 30 Year Fixed Rate Mortgage Loan of $320,000.00 we were able to assess the folowing risk:
Probability of Default(PD): 1.92 %
Loss Given Default (LGD): 56.1 %
Exposure at Default (EAD): $315,714.76
Expected Loss (LD): $5,965.61


For the 15 Year Fixed Rate Mortgage Loan of $320,000.00 we were able to assess the folowing risk:
Probability of Default(PD): 1.92 %
Loss Given Default (LGD): 56.1 %
Exposure at Default (EAD): $315,714.76
Expected Loss (LD): $5,965.61




### Securization (WIP)


Mortgage securization aims to pool mortgages and sell them as securities to investors. This has two benefits:
- It transfer the risk of default to investors
- It frees up capital for additional lending
However, securitization also requires careful risk management to ensure that the underlying mortgages are properly assessed for risk.

